In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
pd.set_option("display.max_columns", 300)

input = 'input/'
output = 'output/'
mapping = 'mapping/'

os.makedirs(output, exist_ok=True)


data_input = glob.glob(os.path.join(input+'*.xlsx'))
data_input

['input\\CAP_data.xlsx']

In [4]:
data_map = glob.glob(os.path.join(mapping+'*csv'))
data_map

['mapping\\RPISIC01_20211031.csv',
 'mapping\\RPISIC01_20211130.csv',
 'mapping\\RPISIC01_20211231.csv',
 'mapping\\RPISIC01_20220131.csv',
 'mapping\\RPISIC01_20220228.csv',
 'mapping\\RPISIC01_20220331.csv',
 'mapping\\RPISIC01_20220430.csv',
 'mapping\\RPISIC01_20220531.csv',
 'mapping\\RPISIC01_20220630.csv',
 'mapping\\RPISIC01_20220731.csv',
 'mapping\\RPISIC01_20220831.csv',
 'mapping\\RPISIC01_20220930.csv',
 'mapping\\RPISIC01_20221031.csv',
 'mapping\\RPISIC01_20221130.csv',
 'mapping\\RPISIC01_20221231.csv']

# CAP_Data range between 10/2021 and 12/2022

In [5]:
def capdata_prepare():
    cap_df = pd.read_excel(data_input[0])

    # rename 
    cap_df.rename(columns={'หมายเลขบัตรประจำตัว':'ID_NO'}, inplace=True)
    cap_df.rename(columns={'เลขที่ใบคำขอ':'APPLICATION_IN'}, inplace=True)

    # drop duplicated ID_NO
    cap_df.drop_duplicates(subset=['ID_NO'], inplace=True)

    # string method --> slice text to get month
    cap_df['Month'] = cap_df['APPLICATION_IN'].str.slice(8,12)

    return cap_df

In [6]:
cap_df = capdata_prepare()

In [7]:
cap_df.head()

,As_of,APPLICATION_IN,ID_NO,Month
0,2021-10-31,AP00001-211004-00002,3950500181876,2110
2,2021-10-31,AP00027-211001-00001,3149900277299,2110
3,2021-10-31,AP00027-211004-00002,3140100522762,2110
4,2021-10-31,AP00029-211001-00001,1102700192688,2110
5,2021-10-31,AP00029-211003-00004,1679900168863,2110


In [8]:
cap_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15804 entries, 0 to 24782
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   As_of           15804 non-null  datetime64[ns]
 1   APPLICATION_IN  15804 non-null  object        
 2   ID_NO           15803 non-null  object        
 3   Month           15804 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 617.3+ KB


In [9]:
cap_df['ID_NO'].value_counts()

3950500181876    1
1840800005920    1
3610500158460    1
3101600417527    1
1102200150625    1
                ..
3100500221565    1
2809700020958    1
1240800157601    1
3840600166730    1
1960800066847    1
Name: ID_NO, Length: 15803, dtype: int64

In [10]:
cap_df['As_of'].value_counts().sort_index()

2021-10-31     724
2021-11-30     881
2021-12-31     674
2022-01-30     651
2022-02-28     700
2022-03-31    1042
2022-04-30     681
2022-05-31     858
2022-06-30    1848
2022-07-31    1258
2022-08-31    1569
2022-09-30    1241
2022-10-31    1174
2022-11-30    1388
2022-12-31    1115
Name: As_of, dtype: int64

In [11]:
cap_df['ID_NO'].dtype

dtype('O')

# CBS as of 12/2022

In [12]:
def cbsdata_prepare():
    cbs_data = pd.read_csv(data_map[-1], encoding="TIS=620", low_memory=False)

    # map ID_NO(CAP) to CBS
    cap_df_ID = cap_df['ID_NO'].unique()
    cbs_data_filt = cbs_data[cbs_data['ID_NO'].isin(cap_df_ID)]

    # filter ACC_OPEN_DATE >= 10/2021 
    cbs_data_filt['ACC_OPEN_DATE'] = pd.to_datetime(cbs_data_filt['ACC_OPEN_DATE'])
    cbs_data_filt = cbs_data_filt[cbs_data_filt['ACC_OPEN_DATE'] >= '2021-10-01']

    # change datetime format
    cbs_data_filt['ACC_OPEN_DATE'] = cbs_data_filt['ACC_OPEN_DATE'].dt.strftime('%d/%m/%Y')
    return cbs_data_filt

In [13]:
cbs_data = cbsdata_prepare()

C:\Users\650010\AppData\Local\Temp\ipykernel_7980\1408289536.py:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  cbs_data_filt['ACC_OPEN_DATE'] = pd.to_datetime(cbs_data_filt['ACC_OPEN_DATE'])
C:\Users\650010\AppData\Local\Temp\ipykernel_7980\1408289536.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbs_data_filt['ACC_OPEN_DATE'] = pd.to_datetime(cbs_data_filt['ACC_OPEN_DATE'])


In [14]:
print(cbs_data.shape)
cbs_data.head()

(6530, 63)


,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,LEDGER_BALANCE,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,NO_OF_DPD,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY
288,31/12/2022,310000590,กูไหรฝ๊ะ หวั่นมาแซะ,9349321289,934,สาขาหาดใหญ่,7200.0,สินเชื่อเพื่ออุปโภค,300000.0,282926.29,B1,CBS,อิสลาม,1_มุสลิม,1,3901100820121,1.0,241041,22.7,1,0,0,0,24/12/2021,24/12/2021,NaN,0,NaN,1Retail,NaN,950000.0,NaN,OLDCIF,สงขลา,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,004-พนักงานและลูกจ้างในสถาบันการเงิน,08-ป.ว.ช.,56.0,"1-<10,000",168M,24/1/2023,0.0,9349321289,24/12/2021,300000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1021.0,นโยบายธนาคาร,413.0,โครงการสินเชื่อ Top Up,241026,12.4,6.0,1-Retail/Consumer,4.65,0,0.0
289,31/12/2022,310000653,การีหมะ แมเร๊าะ,419023216,41,สาขาตันหยงมัส,7100.0,สินเชื่อเพื่อซื้อที่อยู่อาศัย,1057000.0,1041266.87,B1,CBS,อิสลาม,1_มุสลิม,1,3960500673582,1.0,241038,22.7,1,1,0,0,18/08/2022,18/8/2022,NaN,0,NaN,1Retail,NaN,2057000.0,NaN,OLDCIF,นราธิวาส,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,001-ข้าราชการและลูกจ้างในหน่วยงานของรัฐ,03-ปริญญาตรี,46.0,"1-<10,000",216M,31/1/2023,0.0,419023216,18/8/2022,1057000.0,1_PF,2022.0,F,10005.0,สินเชื่อเพื่อไถ่ถอนที่อยู่อาศัย,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,2.10,0,0.0
290,31/12/2022,310000653,การีหมะ แมเร๊าะ,419023224,41,สาขาตันหยงมัส,7200.0,สินเชื่อเพื่ออุปโภค,1000000.0,988458.26,B1,CBS,อิสลาม,1_มุสลิม,1,3960500673582,1.0,241040,22.7,0,0,0,0,18/08/2022,18/8/2022,NaN,0,NaN,1Retail,NaN,2057000.0,NaN,OLDCIF,นราธิวาส,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,001-ข้าราชการและลูกจ้างในหน่วยงานของรัฐ,03-ปริญญาตรี,46.0,"1-<10,000",216M,31/1/2023,0.0,419023224,18/8/2022,1000000.0,1_PF,2022.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,4.75,0,0.0
301,31/12/2022,310000770,กูยะห์รอ อีแต,909018642,90,สาขารือเสาะ นราธิวาส,7200.0,สินเชื่อเพื่ออุปโภค,1500000.0,1488687.17,B1,CBS,อิสลาม,1_มุสลิม,1,1959900067169,1.0,241040,22.7,1,1,0,0,11/11/2022,11/11/2022,NaN,0,NaN,1Retail,NaN,1565000.0,NaN,NEWCIF,นราธิวาส,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,001-ข้าราชการและลูกจ้างในหน่วยงานของรัฐ,03-ปริญญาตรี,36.0,"1-<10,000",240M,31/1/2023,0.0,909018642,11/11/2022,1500000.0,1_PF,2022.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,4.50,0,0.0
302,31/12/2022,310000770,กูยะห์รอ อีแต,909018650,90,สาขารือเสาะ นราธิวาส,7200.0,สินเชื่อเพื่ออุปโภค,65000.0,64367.56,B1,CBS,อิสลาม,1_มุสลิม,1,1959900067169,1.0,241038,22.7,0,0,0,0,11/11/2022,11/11/2022,NaN,0,NaN,1Retail,NaN,1565000.0,NaN,NEWCIF,นราธิวาส,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,001-ข้าราชการและลูกจ้างในหน่วยงานของรัฐ,03-ปริญญาตรี,36.0,"1-<10,000",240M,31/1/2023,0.0,909018650,11/11/2022,65000.0,1_PF,2022.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1090.0,เพื่อชำระเงินสมทบประกันตะกาฟุล,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,4.50,0,0.0


In [15]:
cbs_data['ID_NO'].duplicated().sum()

2041

In [16]:
cbs_data['ACC_ID'].duplicated().sum()

0

In [17]:
cbs_data.duplicated().sum()

0

# Mapping 

- [df_left] = cbs_data
- [df_right] = cbs each month (must data preparation)

process : 
-  [df_left] with "left join" map to [df_right]

In [18]:
def convert_dq(x):
    if (x <= 7):
        return 'DQ_0'
    elif (x >= 8) & (x <= 30):
        return 'DQ_X'
    elif (x >= 31) & (x <= 60):
        return 'DQ_30'
    elif (x >= 61) & (x <= 90):
        return 'DQ_60'
    elif (x >= 91):
        return 'DQ_90'

def convert_code(x):
    if (x <= 7):
        return 0
    elif (x >= 8) & (x <= 30):
        return 1
    elif (x >= 31) & (x <= 60):
        return 2
    elif (x >= 61) & (x <= 90):
        return 3
    elif (x >= 91):
        return 4

In [19]:
data_map_test = data_map[0:3]
data_map_test

['mapping\\RPISIC01_20211031.csv',
 'mapping\\RPISIC01_20211130.csv',
 'mapping\\RPISIC01_20211231.csv']

In [20]:
data_map

['mapping\\RPISIC01_20211031.csv',
 'mapping\\RPISIC01_20211130.csv',
 'mapping\\RPISIC01_20211231.csv',
 'mapping\\RPISIC01_20220131.csv',
 'mapping\\RPISIC01_20220228.csv',
 'mapping\\RPISIC01_20220331.csv',
 'mapping\\RPISIC01_20220430.csv',
 'mapping\\RPISIC01_20220531.csv',
 'mapping\\RPISIC01_20220630.csv',
 'mapping\\RPISIC01_20220731.csv',
 'mapping\\RPISIC01_20220831.csv',
 'mapping\\RPISIC01_20220930.csv',
 'mapping\\RPISIC01_20221031.csv',
 'mapping\\RPISIC01_20221130.csv',
 'mapping\\RPISIC01_20221231.csv']

In [21]:
cap_df_ID = cap_df['ID_NO'].unique()

def mapping_process(cbs_data):  #data_map

    # drop columns: NO_OF_DPD, LEDGER_BALANCE
    df_left = cbs_data.drop(columns=['NO_OF_DPD', 'LEDGER_BALANCE'])
    df_left['BOOKED_(MM/YYYY)'] = np.nan
    df_left['BOOKED_(YYYY-QQ)'] = np.nan

    for file in data_map:
        df_right = pd.read_csv(file, encoding='TIS-620', low_memory=False)

        # map ID_NO(CAP) to df_right
        # df_right = df_right[df_right['ID_NO'].isin(cap_df_ID)]

        # select field 
        df_right = df_right.loc[:, ['ACC_ID', 'NO_OF_DPD', 'LEDGER_BALANCE']]

        # mapping FK="ACC_ID" 
        df_left = df_left.merge(df_right, on=['ACC_ID'], how='left')

        # apply function DQ
        df_left['DQ'] = df_left['NO_OF_DPD'].apply(convert_dq)
        df_left['CODE'] = df_left['NO_OF_DPD'].apply(convert_code)

        # rename
        df_left.rename(columns={'NO_OF_DPD':'NO_OF_DPD_{}'.format(file[17:-6])}, inplace=True)
        df_left.rename(columns={'LEDGER_BALANCE':'LEDGER_BALANCE_{}'.format(file[17:-6])}, inplace=True) 
        df_left.rename(columns={'DQ':'DQ_{}'.format(file[17:-6])}, inplace=True)
        df_left.rename(columns={'CODE':'CODE_{}'.format(file[17:-6])}, inplace=True)


        """
        add book date each ACC_ID
        """

        # find and determine column index: "BOOKED_(MM/YYYY)"
        col_booked = [col for col in df_left.columns.to_list() if col[0:11] == 'BOOKED_(MM/']
        ind_col_booked = [df_left.columns.get_loc(mon) for mon in col_booked]
        
        # scan column "NO_OF_DPD" each loop
        col_dpd = [col for col in df_left.columns.to_list() if col[0:5] == 'NO_OF']
        ind_col_dpd = [df_left.columns.get_loc(mon) for mon in col_dpd]

        # date string format yyyy-mm
        date_str = "{}".format(file[17:-6])

        # condition 
        # col_dpd = 1 --> for first month
        if len(col_dpd) == 1:
            indexrow_notna = df_left[df_left.iloc[:,ind_col_dpd[0]].notna()].index
            df_left.iloc[indexrow_notna, ind_col_booked] = date_str
            # print("have dpd 1 col")

        # col_dpd > 1 --> for not first month
        else:
            ind_col_i_2m = [i for i in list(range(0, len(ind_col_dpd)-1))]
            ind_col_j_2m = [i for i in list(range(0, len(ind_col_dpd)))]
            
            for i, j in zip(ind_col_i_2m, ind_col_i_2m):           
                j = j+2 
                indexrow_notna2 = df_left[df_left.iloc[:,ind_col_dpd[-1]].notna()].index
                indexrow_notna1 = df_left[df_left.iloc[:,ind_col_dpd[-2]].notna()].index                
                index_notna2 = indexrow_notna2.difference(indexrow_notna1) 
                df_left.iloc[index_notna2, ind_col_booked] = date_str
            # print("have > 1 col")

        print(f"mapping file {file} done!")
    
    # string method --> slicing for prepare format MM/YYYY 
    df_left['BOOKED_(MM/YYYY)'] = df_left['BOOKED_(MM/YYYY)'].str[4:6] +"/" + df_left['BOOKED_(MM/YYYY)'].str[0:4]
    # convert to quarter 
    df_left['BOOKED_(YYYY-QQ)'] = df_left['BOOKED_(YYYY-QQ)'].fillna(value = pd.to_datetime(df_left['BOOKED_(MM/YYYY)']).dt.to_period("Q"))

    return df_left

In [22]:
df = mapping_process(cbs_data)

mapping file mapping\RPISIC01_20211031.csv done!
mapping file mapping\RPISIC01_20211130.csv done!
mapping file mapping\RPISIC01_20211231.csv done!
mapping file mapping\RPISIC01_20220131.csv done!
mapping file mapping\RPISIC01_20220228.csv done!
mapping file mapping\RPISIC01_20220331.csv done!
mapping file mapping\RPISIC01_20220430.csv done!
mapping file mapping\RPISIC01_20220531.csv done!
mapping file mapping\RPISIC01_20220630.csv done!
mapping file mapping\RPISIC01_20220731.csv done!
mapping file mapping\RPISIC01_20220831.csv done!
mapping file mapping\RPISIC01_20220930.csv done!
mapping file mapping\RPISIC01_20221031.csv done!
mapping file mapping\RPISIC01_20221130.csv done!
mapping file mapping\RPISIC01_20221231.csv done!


In [23]:
df[df['BOOKED_(MM/YYYY)'].notna()]['BOOKED_(MM/YYYY)'].value_counts().sort_index()

01/2022    171
02/2022    200
03/2022    291
04/2022    282
05/2022    245
06/2022    295
07/2022    534
08/2022    815
09/2022    887
10/2021    193
10/2022    718
11/2021    156
11/2022    860
12/2021    175
12/2022    708
Name: BOOKED_(MM/YYYY), dtype: int64

In [24]:
df['BOOKED_(YYYY-QQ)'].value_counts()

2022Q4    2286
2022Q3    2236
2022Q2     822
2022Q1     662
2021Q4     524
Freq: Q-DEC, Name: BOOKED_(YYYY-QQ), dtype: int64

In [28]:
print(df.shape)
df.head()

(6530, 123)


,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY,BOOKED_(MM/YYYY),BOOKED_(YYYY-QQ),NO_OF_DPD_202110,LEDGER_BALANCE_202110,DQ_202110,CODE_202110,NO_OF_DPD_202111,LEDGER_BALANCE_202111,DQ_202111,CODE_202111,NO_OF_DPD_202112,LEDGER_BALANCE_202112,DQ_202112,CODE_202112,NO_OF_DPD_202201,LEDGER_BALANCE_202201,DQ_202201,CODE_202201,NO_OF_DPD_202202,LEDGER_BALANCE_202202,DQ_202202,CODE_202202,NO_OF_DPD_202203,LEDGER_BALANCE_202203,DQ_202203,CODE_202203,NO_OF_DPD_202204,LEDGER_BALANCE_202204,DQ_202204,CODE_202204,NO_OF_DPD_202205,LEDGER_BALANCE_202205,DQ_202205,CODE_202205,NO_OF_DPD_202206,LEDGER_BALANCE_202206,DQ_202206,CODE_202206,NO_OF_DPD_202207,LEDGER_BALANCE_202207,DQ_202207,CODE_202207,NO_OF_DPD_202208,LEDGER_BALANCE_202208,DQ_202208,CODE_202208,NO_OF_DPD_202209,LEDGER_BALANCE_202209,DQ_202209,CODE_202209,NO_OF_DPD_202210,LEDGER_BALANCE_202210,DQ_202210,CODE_202210,NO_OF_DPD_202211,LEDGER_BALANCE_202211,DQ_202211,CODE_202211,NO_OF_DPD_202212,LEDGER_BALANCE_202212,DQ_202212,CODE_202212
0,31/12/2022,310000590,กูไหรฝ๊ะ หวั่นมาแซะ,9349321289,934,สาขาหาดใหญ่,7200.0,สินเชื่อเพื่ออุปโภค,300000.0,B1,CBS,อิสลาม,1_มุสลิม,1,3901100820121,1.0,241041,22.7,1,0,0,0,24/12/2021,24/12/2021,NaN,0,NaN,1Retail,NaN,950000.0,NaN,OLDCIF,สงขลา,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,004-พนักงานและลูกจ้างในสถาบันการเงิน,08-ป.ว.ช.,56.0,"1-<10,000",168M,24/1/2023,9349321289,24/12/2021,300000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1021.0,นโยบายธนาคาร,413.0,โครงการสินเชื่อ Top Up,241026,12.4,6.0,1-Retail/Consumer,4.65,0,0.0,12/2021,2021Q4,NaN,NaN,None,NaN,NaN,NaN,None,NaN,0.0,300000.0,DQ_0,0.0,0.0,298621.0,DQ_0,0.0,0.0,297237.04,DQ_0,0.0,0.0,295740.32,DQ_0,0.0,0.0,294381.14,DQ_0,0.0,0.0,292910.27,DQ_0,0.0,0.0,291504.87,DQ_0,0.0,0.0,290094.22,DQ_0,0.0,0.0,288643.33,DQ_0,0.0,0.0,287291.57,DQ_0,0.0,0.0,285795.91,DQ_0,0.0,0.0,284329.48,DQ_0,0.0,0.0,282926.29,DQ_0,0
1,31/12/2022,310000653,การีหมะ แมเร๊าะ,419023216,41,สาขาตันหยงมัส,7100.0,สินเชื่อเพื่อซื้อที่อยู่อาศัย,1057000.0,B1,CBS,อิสลาม,1_มุสลิม,1,3960500673582,1.0,241038,22.7,1,1,0,0,18/08/2022,18/8/2022,NaN,0,NaN,1Retail,NaN,2057000.0,NaN,OLDCIF,นราธิวาส,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,001-ข้าราชการและลูกจ้างในหน่วยงานของรัฐ,03-ปริญญาตรี,46.0,"1-<10,000",216M,31/1/2023,419023216,18/8/2022,1057000.0,1_PF,2022.0,F,10005.0,สินเชื่อเพื่อไถ่ถอนที่อยู่อาศัย,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,2.10,0,0.0,08/2022,2022Q3,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,0.0,1057000.00,DQ_0,0.0,0.0,1053714.99,DQ_0,0.0,0.0,1049391.23,DQ_0,0.0,0.0,1049391.23,DQ_0,0.0,0.0,1041266.87,DQ_0,0
2,31/12/2022,310000653,การีหมะ แมเร๊าะ,419023224,41,สาขาตันหยงมัส,7200.0,สินเชื่อเพื่ออุปโภค,1000000.0,B1,CBS,อิสลาม,1_มุสลิม,1,3960500673582,1.0,241040,22.7,0,0,0,0,18/08/2022,18/8/2022,NaN,0,NaN,1Retail,NaN,2057000.0,NaN,OLDCIF,นราธิวาส,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้มีหลักประกัน,001-ข้าราชการและลูกจ้างในหน่วยงานของรัฐ,03-ปริญญาตรี,46.0,"1-<10,000",216M,31/1/2023,419023224,18/8/2022,1000000.0,1_PF,2022.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,

In [234]:
list_acc = df[df['NO_OF_DPD_202111'].notna()]['ACC_ID'].unique()
df[df['ACC_ID'].isin(list_acc)].shape

(349, 75)

## method1

In [352]:
# (have 1 column) MEDTHOD_1
# row
index_notna = df[df['NO_OF_DPD_202110'].notna()].index

# column
col_code = [col for col in df.columns.to_list() if col[0:12] == 'BOOKED_(YYYY']
ind_col = [df.columns.get_loc(mon) for mon in col_code]
print(ind_col)


df.iloc[index_notna, ind_col] = "FILL STR"

[61]


In [353]:
df[df['ACC_ID'].isin(list_acc)]['BOOKED_(YYYY-MM)']

13      FILL STR
23           NaN
56      FILL STR
103     FILL STR
119     FILL STR
          ...   
2609         NaN
2611         NaN
2612         NaN
2613         NaN
2614         NaN
Name: BOOKED_(YYYY-MM), Length: 349, dtype: object

## method2

In [357]:
# test  MEDTHOD_2

# scan columns
col_dpd = [col for col in df.columns.to_list() if col[0:5] == 'NO_OF']
print(col_dpd, '\n')

ind_col_dpd = [df.columns.get_loc(mon) for mon in col_dpd]
print(ind_col_dpd)
# print(ind_col_dpd[0])

print(ind_col_dpd[-2])
print(ind_col_dpd[-1])

['NO_OF_DPD_202110', 'NO_OF_DPD_202111', 'NO_OF_DPD_202112'] 

[63, 67, 71]
67
71


In [273]:
if len(col_dpd) == 1:
    print("have dpd 1 col")
else:
    print("have > 1 col")

have > 1 col


In [358]:
# rows

# 10/2021 - 11/2021
indexrow_notna1 = df[df.iloc[:,ind_col_dpd[-3]].notna()].index

indexrow_notna2 = df[df.iloc[:,ind_col_dpd[-2]].notna()].index
print(len(indexrow_notna2))
print(len(indexrow_notna1))
print("B-A = {}".format(len(indexrow_notna2.difference(indexrow_notna1))))

# B-A
index_notna2 = indexrow_notna2.difference(indexrow_notna1) 

349
193
B-A = 156


In [359]:
# fill data

df.iloc[index_notna2, ind_col] = "FILL STR 2"
df[df['NO_OF_DPD_202111'].notna()]['BOOKED_(YYYY-MM)'].value_counts()

FILL STR      193
FILL STR 2    156
Name: BOOKED_(YYYY-MM), dtype: int64

In [333]:
# 3 months calculation ---> check list index below here!!  
print("Columns for 3M")
ind_col_i_2m = [i for i in list(range(0, len(ind_col_dpd)-1))]
ind_col_j_2m = [i for i in list(range(0, len(ind_col_dpd)))]

for i, j in zip(ind_col_i_2m, ind_col_i_2m):
    j = j+2                                     # range for cal
    print(ind_col_dpd[i:j])

Columns for 3M
[63, 67]
[67, 71]


In [360]:
for i, j in zip(ind_col_i_2m, ind_col_j_2m):
    j = j+2                                     # range for cal
    print(ind_col_dpd[i:j][-1])
    print(ind_col_dpd[i:j][-2])

    indexrow_notna2 = df[df.iloc[:,ind_col_dpd[-1]].notna()].index
    indexrow_notna1 = df[df.iloc[:,ind_col_dpd[0]].notna()].index
    # print(len(indexrow_notna2))
    # print(len(indexrow_notna1))
    # print("B-A = {}".format(len(indexrow_notna2.difference(indexrow_notna1))))

    index_notna2 = indexrow_notna2.difference(indexrow_notna1) 
    
    # fill data
    df.iloc[index_notna2, ind_col] = "FILL STR 2"

67
63
71
67


In [306]:
df[df.iloc[:,ind_col_dpd[-3]].notna()].index

Int64Index([  13,   56,  103,  119,  170,  171,  188,  189,  219,  220,
            ...
            2487, 2488, 2489, 2490, 2491, 2492, 2494, 2496, 2497, 2498],
           dtype='int64', length=193)

In [30]:
# df[df['NO_OF_DPD_202111'].notna()].shape

(349, 69)

In [ ]:
df[df['NO_OF_DPD_202212'].notna()].shape

In [514]:
cbs_data[cbs_data['ACC_ID'] == 3669040498]

,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,LEDGER_BALANCE,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,NO_OF_DPD,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY
569,31/12/2022,310004798,การีมะห์ มะเกะ,3669040498,366,สาขาถนนวิจิตรไชยบูลย์ นราธิวาส,7200.0,สินเชื่อเพื่ออุปโภค,690000.0,658813.71,B1,CBS,อิสลาม,1_มุสลิม,1,1950600013600,1.0,241041,22.7,1,1,0,0,29/10/2021,29/10/2021,NaN,0,NaN,1Retail,NaN,690000.0,NaN,OLDCIF,นราธิวาส,7 ภาคใต้,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,001-ข้าราชการและลูกจ้างในหน่วยงานของรัฐ,03-ปริญญาตรี,37.0,"1-<10,000",180M,31/1/2023,0.0,3669040498,29/10/2021,690000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0,0.0


In [515]:
df[df['NO_OF_DPD_202201'].notna()].shape

(695, 121)

# Export data

In [29]:
# df.to_excel(output+"CAP_CBS_Performace.xlsx", index=False, encoding="TIS-620")
# df.to_csv(output+"CAP_CBS_Performace.txt", index=False, encoding="TIS-620", sep="|")

# Check datatype

In [506]:
for i in data_map:
    data = pd.read_csv(i, encoding="TIS=620", low_memory=False)
    print('"{}" ACC_ID {}, LEDGER_BALANCE {}, ID_NO {}, na {}'.format( i[17:-6], 
                                                    data['ACC_ID'].dtype, 
                                                    data['LEDGER_BALANCE'].dtype,
                                                    data['ID_NO'].dtype, 
                                                    data['ACC_ID'].isna().sum() ) )

"202110" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202111" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202112" ACC_ID float64, LEDGER_BALANCE float64, ID_NO object, na 0
"202201" ACC_ID float64, LEDGER_BALANCE int64, ID_NO object, na 0
"202202" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202203" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202204" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202205" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202206" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202207" ACC_ID float64, LEDGER_BALANCE float64, ID_NO object, na 0
"202208" ACC_ID float64, LEDGER_BALANCE float64, ID_NO object, na 0
"202209" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202210" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202211" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0
"202212" ACC_ID int64, LEDGER_BALANCE float64, ID_NO object, na 0


# Check NA after mapping

In [516]:
col_code = [col for col in df.columns.to_list() if col[0:5] == 'NO_OF' ]
ind_col = [df.columns.get_loc(mon) for mon in col_code]
for month in col_code:
    print("Column: {} ({})".format(month, df.columns.get_loc(month)))
print('')
print('index', ind_col)

Column: NO_OF_DPD_202110 (61)
Column: NO_OF_DPD_202111 (65)
Column: NO_OF_DPD_202112 (69)
Column: NO_OF_DPD_202201 (73)
Column: NO_OF_DPD_202202 (77)
Column: NO_OF_DPD_202203 (81)
Column: NO_OF_DPD_202204 (85)
Column: NO_OF_DPD_202205 (89)
Column: NO_OF_DPD_202206 (93)
Column: NO_OF_DPD_202207 (97)
Column: NO_OF_DPD_202208 (101)
Column: NO_OF_DPD_202209 (105)
Column: NO_OF_DPD_202210 (109)
Column: NO_OF_DPD_202211 (113)
Column: NO_OF_DPD_202212 (117)

index [61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101, 105, 109, 113, 117]


In [517]:
for index, col, dpd in zip(ind_col, col_code, ind_col):
    print("not NA rows -- {} = {} , sum dpd = {}".format(col, 
                                                        df.iloc[:, index].notna().sum(), 
                                                        np.sum(df.iloc[:,dpd], axis=0)))

not NA rows -- NO_OF_DPD_202110 = 193 , sum dpd = 0.0
not NA rows -- NO_OF_DPD_202111 = 349 , sum dpd = 5.0
not NA rows -- NO_OF_DPD_202112 = 524 , sum dpd = 14.0
not NA rows -- NO_OF_DPD_202201 = 695 , sum dpd = 0.0
not NA rows -- NO_OF_DPD_202202 = 895 , sum dpd = 33.0
not NA rows -- NO_OF_DPD_202203 = 1186 , sum dpd = 72.0
not NA rows -- NO_OF_DPD_202204 = 1468 , sum dpd = 113.0
not NA rows -- NO_OF_DPD_202205 = 1713 , sum dpd = 210.0
not NA rows -- NO_OF_DPD_202206 = 2008 , sum dpd = 446.0
not NA rows -- NO_OF_DPD_202207 = 2542 , sum dpd = 753.0
not NA rows -- NO_OF_DPD_202208 = 3357 , sum dpd = 1037.0
not NA rows -- NO_OF_DPD_202209 = 4244 , sum dpd = 1410.0
not NA rows -- NO_OF_DPD_202210 = 4962 , sum dpd = 1838.0
not NA rows -- NO_OF_DPD_202211 = 5822 , sum dpd = 2032.0
not NA rows -- NO_OF_DPD_202212 = 6530 , sum dpd = 2564.0


In [518]:
np.sum(df['LEDGER_BALANCE_202110'], axis=0)

79746027.62

In [519]:
np.sum(df['LEDGER_BALANCE_202111'], axis=0)

132543799.4